#### Links

https://www.hockey-reference.com/draft/NHL_2000_entry.html  
Hockey-Reference is where the playoff and draft data came from.  
http://www.hockeyabstract.com/testimonials/nhlplayerdata1967-2013  
Hockey Abstract is where the player data came from.  

In [1]:
import pandas as pd
import os
import pymysql.cursors

In [2]:
# load playoff data
playoffs = pd.read_csv("nhlplayoffs.csv")

In [3]:
# load player data
players= pd.read_excel("NHLPlayerData1967_2018.xlsx", sheet_name="Player Data")

In [29]:
# load draft data
drafts = {}
for f in os.listdir("drafts"):
    drafts[f.split(".")[0]] = pd.read_csv("drafts\{}".format(f))

In [5]:
# get teams printed
teams = set()
for row in playoffs.itertuples():
    teams.add(playoffs.at[row[0], 'Champion'])
    teams.add(playoffs.at[row[0], 'Runner-Up'])

In [6]:
teams_ac = set()
for row in players.itertuples():
    teams_ac.add(players.at[row[0], 'Team'])

In [7]:
for k, v in drafts.items():
    for row in v.itertuples():
        teams.add(v.at[row[0], 'Team'])

In [8]:
print(teams)
print(teams_ac)

{nan, 'Colorado Avalanche', 'Edmonton Oilers', 'Buffalo Sabres', 'Nashville Predators', 'New York Islanders', 'Detroit Red Wings', 'Calgary Flames', 'Los Angeles Kings', 'Minnesota North Stars', 'Florida Panthers', 'Dallas Stars', 'New Jersey Devils', 'Atlanta Thrashers', 'Ottawa Senators', 'Columbus Blue Jackets', 'Toronto Maple Leafs', 'Arizona Coyotes', 'Season canceled', 'New York Rangers', 'Vancouver Canucks', 'Winnipeg Jets', 'Vegas Golden Knights', 'San Jose Sharks', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Montreal Canadiens', 'Tampa Bay Lightning', 'Boston Bruins', 'Chicago Blackhawks', 'St. Louis Blues', 'Minnesota Wild', 'Washington Capitals', 'Mighty Ducks of Anaheim', 'Phoenix Coyotes', 'Carolina Hurricanes', 'Anaheim Ducks'}
{nan, 'WSH', 'CBJ', 'WIN', 'TBL', 'PIT', 'HAR', 'BOS', 'COL', 'MIN', 'CHI', 'PHX', 'CLR', 'SJS', 'TOT', 'ANA', 'ATL', 'CAR', 'KCS', 'VAN', 'CGY', 'NYR', 'PHI', 'NYI', 'CBH', 'WPG', 'MDA', 'STL', 'ATF', 'CGS', 'OAK', 'BUF', 'CLE', 'MNS', 'QUE', '

In [9]:
# with this I can make the list
# since we are only going back to the year 2000, I can discount any teams not already in the league.
# I can also discount Seattle, as they have not started playing yet.
# I cannot count VGK either, since they were built in an expansion draft
# note that the atlanta thrashers franchise became the winnipeg jets in 2011

teams = {'DAL': 'Dallas Stars',
         'FLA': 'Florida Panthers',
         'LAK': 'Los Angeles Kings',
         'PHX': 'Phoenix Coyotes',
         'WPG': 'Winnipeg Jets',
         'MIN': 'Minnesota Wild',
         'VAN': 'Vancouver Canucks',
         'CHI': 'Chicago Blackhawks',
         'WSH': 'Washington Capitals',
         'PIT': 'Pittsburgh Penguins',
         'ARI': 'Arizona Coyotes',
         'NJD': 'New Jersey Devils',
         'ANA': 'Anaheim Ducks',
         'EDM': 'Edmonton Oilers',
         'NSH': 'Nashville Predators',
         'CGY': 'Calgary Flames',
         'PHI': 'Philadelphia Flyers',
         'TBL': 'Tampa Bay Lightning',
         'SJS': 'San Jose Sharks',
         'BUF': 'Buffalo Sabres',
         'CBJ': 'Columbus Blue Jackets',
         'NYI': 'New York Islanders',
         'NYR': 'New York Rangers',
         'TOR': 'Toronto Maple Leafs',
         'CAR': 'Carolina Hurricanes',
         'STL': 'St. Louis Blues',
         'COL': 'Colorado Avalanche',
         'DET': 'Detroit Red Wings',
         'BOS': 'Boston Bruins',
         'OTT': 'Ottawa Senators',
         'MTL': 'Montreal Canadiens',
         'VGK': 'Vegas Golden Knights',
         'ATL': 'Atlanta Thrashers',
         'MDA': 'Mighty Ducks of Anaheim' # Anaheim Duck
        }

In [10]:
# send it all to a sql database?
"""
Tables
Franchises - ID, Acronym, Name (so we can have multiple teams with the same franchise)
PlayerSeason -  
PlayoffFinals - ID, Season, Champion, RunnerUp
SeasonConnection - Season1, Season2 --> where season1 is a single year and season2 is two years year1-year2 (year2 match to season1)
Drafts - ID, Position, Team, PlayerId

Functions
GetRoster
For a given team in a given season, create their roster based on the year.
Can identify which of those players were drafted by that team.


"""

'\nTables\nFranchises - ID, Acronym, Name (so we can have multiple teams with the same franchise)\nPlayerSeason - ID, Season, Team\nPlayoffFinals - ID, Season, Champion, RunnerUp\nSeasonConnection - Season1, Season2 --> where season1 is a single year and season2 is two years year1-year2 (year2 match to season1)\nDrafts - ID, Position, Team, PlayerId\n\nFunctions\nGetRoster\nFor a given team in a given season, create their roster based on the year.\nCan identify which of those players were drafted by that team.\n\n\n'

In [10]:
# adding data to the sql data
import config as cfg
def connect_to_db():
    """
    Use this method to open connection to the database
    """
    db_conn = pymysql.connect(host=cfg.host,
                              user=cfg.user,
                              password=cfg.pw,
                              db=cfg.db,
                              charset='utf8mb4',
                              cursorclass=pymysql.cursors.DictCursor)

    
    return db_conn

def submit_query(db_conn, query, isreturn=False, values=None):
    """
    Submit a SQL query. Takes in a query via query, and if that is a query to return data,
    set isreturn to True
    """
    cur = db_conn.cursor()

    if values is None:
        cur.execute(query)
    else:
        cur.execute(query, values)
    
    if isreturn:
        return cur.fetchall()
    else:
        return

In [11]:
def insertion(values_dict, table_name):
	"""
	Since we are adding data to a values dict, we can create insertion queries via function
	"""
	front = 'INSERT INTO {}.{}'.format(cfg.schema, table_name)
	
	
	# after front define columns and values
	cols = "" # key
	vals = "" # %s for security
	form_vals = [] # list of actual values, to be converted to tuple
	first = True
	for k, v in values_dict.items():
		if first:
			cols = "{}{}".format(cols, k)
			vals = "{}%s".format(vals, v)
			form_vals.append(v)
			first = False
		else:
			cols = "{},{}".format(cols, k)
			vals = "{},%s".format(vals, v)
			form_vals.append(v)
	# once done, enclose in brackets
	cols = "({})".format(cols)
	vals = "({})".format(vals)
	
	return "{} {} VALUES {};".format(front, cols, vals), tuple(form_vals)

In [23]:
# first we need to create the FranchiseIDs. Teams like WPG/ATL and ANA/MDA have multiple teams under a single franchise
franchise_ids = []
hold1 = {"WPG":-1, "ATL":-1}
hold2 = {"ANA":-1, "MDA":-1}
#hold3 = {"QC":-1, "COL":-1}
#hold4 = {"DAL":-1, "MNS":-1}
#hold5 = {"HFD":-1, "CAR":-1}

counter = 0
for k, v in teams.items():
    if k in list(hold1.keys()):
        if hold1['ATL'] > -1 and hold1['WPG'] == -1:
            # assign atl value to wpg
            franchise_ids.append((hold1["ATL"], k, v))
        elif hold1['ATL'] == -1 and hold1['WPG'] > -1:
            # assign wpg value to atl
            franchise_ids.append((hold1["WPG"], k, v))            
        else:
            # assign this franchise the current value and update the hold
            franchise_ids.append((counter, k, v))
            hold1[k] = counter
    elif k in list(hold2.keys()):
        if hold2['ANA'] > -1 and hold2['MDA'] == -1:
            # assign ana value to mda
            franchise_ids.append((hold2["ANA"], k, v))
        elif hold2['ANA'] == -1 and hold2['MDA'] > -1:
            # assign mda value to ana
            franchise_ids.append((hold2["MDA"], k, v))            
        else:
            # assign this franchise the current value and update the hold
            franchise_ids.append((counter, k, v))
            hold2[k] = counter        
    else:        
        franchise_ids.append((counter, k, v))
    counter += 1

# do it manually for some
franchise_ids.append((0, 'MNS', 'Minnesota North Stars'))
franchise_ids.append((24, 'HFD', 'Hartford Whalers'))
franchise_ids.append((26, 'QCN', 'Quebec Nordiques'))
franchise_ids.append((3, 'WIN', 'Winnipeg Jets')) # pre-2000
print(franchise_ids)
# franchise ids can now be added to the database

[(0, 'DAL', 'Dallas Stars'), (1, 'FLA', 'Florida Panthers'), (2, 'LAK', 'Los Angeles Kings'), (3, 'PHX', 'Phoenix Coyotes'), (4, 'WPG', 'Winnipeg Jets'), (5, 'MIN', 'Minnesota Wild'), (6, 'VAN', 'Vancouver Canucks'), (7, 'CHI', 'Chicago Blackhawks'), (8, 'WSH', 'Washington Capitals'), (9, 'PIT', 'Pittsburgh Penguins'), (10, 'ARI', 'Arizona Coyotes'), (11, 'NJD', 'New Jersey Devils'), (12, 'ANA', 'Anaheim Ducks'), (13, 'EDM', 'Edmonton Oilers'), (14, 'NSH', 'Nashville Predators'), (15, 'CGY', 'Calgary Flames'), (16, 'PHI', 'Philadelphia Flyers'), (17, 'TBL', 'Tampa Bay Lightning'), (18, 'SJS', 'San Jose Sharks'), (19, 'BUF', 'Buffalo Sabres'), (20, 'CBJ', 'Columbus Blue Jackets'), (21, 'NYI', 'New York Islanders'), (22, 'NYR', 'New York Rangers'), (23, 'TOR', 'Toronto Maple Leafs'), (24, 'CAR', 'Carolina Hurricanes'), (25, 'STL', 'St. Louis Blues'), (26, 'COL', 'Colorado Avalanche'), (27, 'DET', 'Detroit Red Wings'), (28, 'BOS', 'Boston Bruins'), (29, 'OTT', 'Ottawa Senators'), (30, 'MT

In [24]:
# loop over franchise_ids and append to the db
db_conn = connect_to_db()
for i in range(len(franchise_ids)):
    row = {}
    row ['FranchiseID'] = franchise_ids[i][0]
    row ['Acronym'] = franchise_ids[i][1]
    row ['TeamName'] = franchise_ids[i][2]
    q, v = insertion(row, 'Franchises')
    print(q, v)
    cur = db_conn.cursor()
    cur.execute(q, v)
    db_conn.commit()
    #submit_query(db_conn, q, values=v)

db_conn.close()

INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (0, 'DAL', 'Dallas Stars')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (1, 'FLA', 'Florida Panthers')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (2, 'LAK', 'Los Angeles Kings')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (3, 'PHX', 'Phoenix Coyotes')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (4, 'WPG', 'Winnipeg Jets')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (5, 'MIN', 'Minnesota Wild')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (6, 'VAN', 'Vancouver Canucks')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (7, 'CHI', 'Chicago Blackhawks')
INSERT INTO nhldrafts.Franchises (FranchiseID,Acronym,TeamName) VALUES (%s,%s,%s); (8, 'WSH', 

In [16]:
# generate the playoffs table using the unique franchise ids
# the teams listed are the teams in index 2 of the franchise_ids tuples
playoff_rows = []

unique_teams = [[],[], []]
for team in franchise_ids:
    unique_teams[0].append(team[0])
    unique_teams[1].append(team[2])
    unique_teams[2].append(team[1])

for row in playoffs.itertuples():
    # print(row.Champion, row._4)
    winner = row.Champion
    loser = row._4
    year = row.Year
    
    win_ind = -1
    los_ind = -1
    
    
    if winner in unique_teams[1]:
        win_ind = unique_teams[1].index(winner)
    if loser in unique_teams[1]:
        los_ind = unique_teams[1].index(loser)
        
    if win_ind > -1 and los_ind > -1:
        _row = {} # ID, Season, Champion, RunnerUp
        _row['Season'] = year
        _row['Champion'] = unique_teams[0][win_ind]
        _row['RunnerUp'] = unique_teams[0][los_ind]
        
        playoff_rows.append(_row)
        
print(playoff_rows)

[{'Season': 2020, 'Champion': 17, 'RunnerUp': 0}, {'Season': 2019, 'Champion': 25, 'RunnerUp': 28}, {'Season': 2018, 'Champion': 8, 'RunnerUp': 31}, {'Season': 2017, 'Champion': 9, 'RunnerUp': 14}, {'Season': 2016, 'Champion': 9, 'RunnerUp': 18}, {'Season': 2015, 'Champion': 7, 'RunnerUp': 17}, {'Season': 2014, 'Champion': 2, 'RunnerUp': 22}, {'Season': 2013, 'Champion': 7, 'RunnerUp': 28}, {'Season': 2012, 'Champion': 2, 'RunnerUp': 11}, {'Season': 2011, 'Champion': 28, 'RunnerUp': 6}, {'Season': 2010, 'Champion': 7, 'RunnerUp': 16}, {'Season': 2009, 'Champion': 9, 'RunnerUp': 27}, {'Season': 2008, 'Champion': 27, 'RunnerUp': 9}, {'Season': 2007, 'Champion': 12, 'RunnerUp': 29}, {'Season': 2006, 'Champion': 24, 'RunnerUp': 13}, {'Season': 2004, 'Champion': 17, 'RunnerUp': 15}, {'Season': 2003, 'Champion': 11, 'RunnerUp': 12}, {'Season': 2002, 'Champion': 27, 'RunnerUp': 24}, {'Season': 2001, 'Champion': 26, 'RunnerUp': 11}, {'Season': 2000, 'Champion': 11, 'RunnerUp': 0}, {'Season': 1

In [17]:
# loop over playoff seasons and append to the db
db_conn = connect_to_db()
for i in range(len(playoff_rows)):
    q, v = insertion(playoff_rows[i], 'PlayoffFinals')
    cur = db_conn.cursor()
    cur.execute(q, v)
    db_conn.commit()
    # submit_query(db_conn, q, values=v)
db_conn.close()

In [14]:
# updating table players   Season, Team --> connect Team to Franchise ID again
# player should be reset to FirstName LAstName
print(players.head())

  Last Name First Name   Season   Age Team Pos  GP  G   A  PTS  ...  SHG  \
0     Aalto      Antti  1997-98  22.0  MDA   C   3  0   0    0  ...    0   
1     Aalto      Antti  1998-99  23.0  MDA   C  73  3   5    8  ...    0   
2     Aalto      Antti  1999-00  24.0  MDA   C  63  7  11   18  ...    0   
3     Aalto      Antti  2000-01  25.0  MDA   C  12  1   1    2  ...    0   
4    Abbott    Spencer  2013-14  25.0  TOR  RW   1  0   0    0  ...    0   

   Shot    TOI  OPS  DPS   PS  OGVT  DGVT  SGVT  GVT  
0     1    0.0 -0.1  0.0 -0.1  -0.2  -0.1   0.0 -0.2  
1    61  685.0 -0.7  0.3 -0.3  -1.7  -0.5   0.0 -2.2  
2   102  830.0  0.3  0.4  0.7   0.2   1.1   0.0  1.3  
3    18  127.0  0.0  0.2  0.2  -0.2   0.4   0.0  0.2  
4     2    5.0  0.0 -0.1 -0.1  -0.2  -0.2   0.0 -0.4  

[5 rows x 24 columns]


In [22]:
db_conn = connect_to_db()

for row in players.itertuples():
    player_row = {}

    player = "{} {}".format(row[2], row[1])
    # create season as the final 2 digits from the Season column, with the addition of either 19 or 20 in front
    season2dig = int(row.Season[-2:])
    # anything above 20 will be in the 20th century (prefix 19) otherwise 21st cnetury (prefix 20)
    if season2dig > 20: 
        season = season2dig + 1900
    else:
        season = season2dig + 2000
    team_ac = row.Team
    team_id = -1
    # get team id
    if team_ac in unique_teams[2]:
        _ind = unique_teams[2].index(team_ac)
        team_id = unique_teams[0][_ind]
    
    player_row['Player'] = player
    player_row['Season'] = season
    player_row['Team'] = team_id
    if team_id > -1:
        q, v = insertion(player_row, 'PlayerSeason')
        cur = db_conn.cursor()
        cur.execute(q, v)
        db_conn.commit()    

db_conn.close()

In [27]:
# loading draft results in
print(drafts['2000'].head())

   Overall                   Team            Player Nat. Pos   Age      To  \
0      1.0     New York Islanders     Rick DiPietro   US   G  18.0  2013.0   
1      2.0      Atlanta Thrashers      Dany Heatley   DE  LW  19.0  2015.0   
2      3.0         Minnesota Wild    Marian Gaborik   SK  RW  18.0  2018.0   
3      4.0  Columbus Blue Jackets  Rostislav Klesla   CZ   D  18.0  2014.0   
4      5.0     New York Islanders      Raffi Torres   CA  LW  18.0  2014.0   

                 Amateur Team      GP      G  ...    PTS   +/-    PIM   GP.1  \
0  Boston University (H-East)   318.0    0.0  ...   19.0   0.0  129.0  318.0   
1            Wisconsin (WCHA)   869.0  372.0  ...  791.0  38.0  620.0    NaN   
2    Dukla Trencin (Slovakia)  1035.0  407.0  ...  815.0  95.0  492.0    NaN   
3              Brampton (OHL)   659.0   48.0  ...  159.0 -46.0  620.0    NaN   
4              Brampton (OHL)   635.0  137.0  ...  260.0   2.0  497.0    NaN   

       W      L   T/O    SV%   GAA     PS  
0  130

In [33]:
db_conn = connect_to_db()
for k, v in drafts.items():
    for row in v.itertuples():
        player_row = {}

        season = k
        team_ac = row.Team
        team_id = -1
        # get team id
        if team_ac in unique_teams[1]:
            _ind = unique_teams[1].index(team_ac)
            team_id = unique_teams[0][_ind]
        try:
            player_row['DraftPosition'] = int(row.Overall)
        except:
            player_row['DraftPosition'] = 999
        player_row['Team'] = team_id
        try:
            player_row['Player'] = row.Player.split("\\")[0]
        except:
            player_row['Player'] = "Unknown Player"
        player_row['Season'] = season
        if team_id > -1:
            q, v = insertion(player_row, 'Drafts')
            cur = db_conn.cursor()
            cur.execute(q, v)
            db_conn.commit()    
    
db_conn.close()